In [72]:
from Base.bp2DBox import Box
from Base.bp2DPnt import Point


class Bin:

    def __init__(self, w, h, use_bottom_left = False):
        self.w = w
        self.h = h
        self.area = w * h  # box, i.e. rectangle, to be packed
        self.pnts_open = None
        
        if use_bottom_left:
            self.bottom_left_points = [Point(0, 0)]
        else:
            self.bottom_left_points = None
        
        
        self.init_pnts_open()  # list of rectangles still to be placed
        self.boxes_stored = []
        self.bounding_box = Box(w, h, Point(0, 0))

    def init_pnts_open(self):
        # self.pnts_open = {Point(i, j) for i in range(self.w) for j in range(self.h)}
        self.pnts_open = [Point(i, j) for i in range(self.w) for j in range(self.h)]
    
# STUFF WITH BOTTOM-LEFT POINTS STARTS HERE
    
    
    def get_bottom_left_points_for_box(self, box: Box):
        '''
        return:
        1: get suitable points for a box, sorted by cost function
        2: area of new bbox / area of old bbox for each position, as cost-function. To be changed 
        somehow need to consider free slots in bbox
        '''
        for 
        
    def update_bottom_left_points(self, box: Box, pnt: Point):
        '''
        !!!! update by Sergei
        Try to place box on bottom-left point. If point is not correct, then this box loses all functions,
        connected to bottom-left points
        
        
        WRONG!!! need to more smart approach
        '''
        if not self.bottom_left_points:
            return
        if pnt in self.bottom_left_points:
            ind = self.bottom_left_points.index(pnt)
            ### box need already moved to new position
            ### tricky thing, tr - top right corner, but not in rectangle!!
            new_x = box.tr.get_x()
            new_y = box.tr.get_y()
            new_points = []
            if new_y < self.h:
                if ind == 0:
                    new_points.append(Point(pnt.get_x(), new_y))
                elif new_y < self.bottom_left_points[ind - 1].get_y():
                    new_points.append(Point(pnt.get_x(), new_y))
                    
            if new_x < self.w:
                if ind == len(self.bottom_left_points) - 1:
                    new_points.append(Point(new_x, pnt.get_y()))
                elif new_x < self.bottom_left_points[ind + 1].get_x():
                    new_points.append(Point(new_x, pnt.get_y()))
                    
            self.bottom_left_points = self.bottom_left_points[:ind] + new_points + self.bottom_left_points[ind + 1:]
            
        else:
            # wrong point
            self.bottom_left_points = None

    def can_place_box_at_pnt(self, box: Box, pnt: Point) -> bool:
        '''Check if Box box can be placed at Point pnt.'''
        if pnt not in self.pnts_open:
            return False
        if box.get_a() > len(self.pnts_open):
            return False

        old_position = box.bl.copy()
        box.move(pnt)

        if not self.bounding_box.contains_rectangle(box):
            box.move(old_position)
            return False
        for bs in self.boxes_stored:
            if bs.overlap(box) > 0:
                box.move(old_position)
                return False
        box.move(old_position)
        return True

    def place_box_at_pnt(self, box: Box, pnt: Point) -> bool:
        '''Place Box box at Point pnt. 
        If the operation is feasible, the function modifies the set of open points and the list of stored boxes and returns True. Otherwise it does nothing and returns False.
        '''
        if self.can_place_box_at_pnt(box, pnt):
            box.move(pnt)
            self.boxes_stored.append(box)
            self.remove_open_pnts(box)
            return True
        else:
            return False

    def get_pnts_open(self):
        return self.pnts_open

    def remove_box(self, box):
        assert box in self.boxes_stored
        self.add_open_pnts(box)
        self.boxes_stored.remove(box)

    def add_open_pnts(self, box: Box):  # TODO shuffles the order of open points if a box is added and then removed
        self.pnts_open.extend(box.get_interior_points())

    def remove_open_pnts(self, box: Box):
        for bp in box.get_interior_points():
            self.pnts_open.remove(bp)

    def capacity_available(self):
        return len(self.pnts_open)

    def get_corner(self, c: str):
        return self.bounding_box.get_corner(c)


if __name__ == '__main__':
    pass


In [84]:
test = Bin(10, 10, True)

In [85]:
test.update_bottom_left_points(Box(9, 9), Point(0, 0))

In [86]:
test.bottom_left_points

[(0, 9), (9, 0)]

In [76]:
box = Box(1, 10)
point = Point(3, 0)
box.move(point)
test.update_bottom_left_points(box, point)

In [77]:
test.bottom_left_points

[(0, 1), (4, 0)]